# <strong><font color=#A52A2A></span> NLP_HW2 </strong>
Dr Asgari - Spring 2023
<br>
98101566 -> Mohammadreza Daviran
<br>
98171007 -> Nona Ghazizadeh
<br>
98170668 -> Sara Azarnoush
<br>


<strong><font color=#B22222></span> Please run this notebook on google colab! </strong>

#Find illegal words in a text

- Requirements
  - Download libraries
  - Download persian fasttext model
  - Import libraries
- Preprocess the data
  - Normalizer and Lemmatizer
  - Download abbreviations
  - Read abbreviation
  - Numbers
  - Read persian fasttext model
- Process the data
  - Find persian words
  - Find similar words in context
  - Print illegal words
  - Find words ignoring spaces
  - Print illegal words considering the spaces
- Test
  - Test without space in each word
  - Test with space in each word
  - Test with abbreviation in text
  - Test with normalizer
  - Test with lemmatizer
  - Test with context

# Requirements
------

Download libraries

In [ ]:
!pip install hazm
!pip install gdown
!pip install fasttext
!pip install Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=9483f6e37abc1bc7605b21e84565f6f672e378548bd320e1389a0c4a3db874f2
  Stored in directory: /root/.cache/pip/wheels/ac/62/f6/88933dadcd64a1614894614aa68cf57c4b8e5256acb650b1f1
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp39-cp39-linux_x86_64.whl size=180375 sha256=95ae50027f770842c750c1afe25a73d9b59967ef8d39e906fa071a634611960d
  Stored in directory: /root/.cache/pip/wheels/8e/ff/82/9326b96f96f47472e02c453697b225813e4650c0ed4df2cd49
Successfully built nltk libwapiti
  Attempting unin

Download persian fasttext model

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz
!gzip -d cc.fa.300.bin.gz

--2023-04-03 11:28:58--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.162.108, 108.157.162.83, 108.157.162.35, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.162.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4502524724 (4.2G) [application/octet-stream]
Saving to: ‘cc.fa.300.bin.gz’

cc.fa.300.bin.gz    100%[===================>]   4.19G   120MB/s    in 34s     

2023-04-03 11:29:33 (125 MB/s) - ‘cc.fa.300.bin.gz’ saved [4502524724/4502524724]



import libraries

In [ ]:
from __future__ import unicode_literals
import re
from hazm import *
import json
import gdown
import fasttext
import Levenshtein

# Load and initialize our requirments
------
Normalizer and lemmatizer 

In [ ]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()

Download abbreviations

In [ ]:
url = "https://drive.google.com/uc?id=12QXnsKtysJzs9r0W-ONMSAMnNuxS8hAa"
output = "abbreviation.json"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=12QXnsKtysJzs9r0W-ONMSAMnNuxS8hAa
To: /content/abbreviation.json
100%|██████████| 14.5k/14.5k [00:00<00:00, 15.5MB/s]


'abbreviation.json'

Read abbreviation

In [ ]:
with open('abbreviation.json', 'r') as file:
    abbreviation = json.load(file)

Numbers

In [ ]:
persian_numbers = '۱۲۳۴۵۶۷۸۹۰'
english_numbers = '1234567890'
english_trans = str.maketrans(persian_numbers, english_numbers)

Load persian fasttext model

In [ ]:
model = fasttext.load_model('cc.fa.300.bin')

# Process the data
-----


Find persian word

In [ ]:
def find_persian(string: str):
    return "".join(re.findall(r"[\u0600-\u06FF]+", string))

Find cotextually similar words to given word

In [ ]:
def find_similar_words_in_context(given_word, k_num=30):
    similar_words = model.get_nearest_neighbors(given_word, k=k_num)
    founded_words = []
    for similarity, word in similar_words:
        if given_word not in word and Levenshtein.distance(word, given_word) > 3 and similarity > 0.5:
            founded_words.append(word)
    return founded_words

Join and print illegal word

In [ ]:
def print_illegal_words(input, ill_words):
  if len(ill_words) > 0:
    print('Without any space between each word: \n')
  for w in ill_words:
    complete_word = ill_words[w]
    index = input.find(complete_word)
    print('word: \"' + w + '\"')
    print(f'span: ({index}, {index + len(complete_word)})\n')

Find and print words without spaces

In [ ]:
def find_words_ignoring_spaces_2(splitted_input, transfered_words):
  found_spaced_ill_words = {}
  for i in range(len(transfered_words) - 1):
    w1 = transfered_words[i]
    w2 = transfered_words[i + 1]
    word = w1 + w2
    if word in illegal_words:
      found_spaced_ill_words[word] = input[input.find(splitted_input[i]): input.find(splitted_input[i + 1])] + w2
  return found_spaced_ill_words

In [ ]:
def find_words_ignoring_spaces(splitted_input, transfered_words, indices):
  found_spaced_ill_words = {}
  
  text = "".join(transfered_words)
  for word in illegal_words:
    range = []
    word = word.strip()
    f = text.find(word)
    if f > 0:
      flag = 0
      for counter, index in enumerate(indices):
        if index[0] == f:
          f = f + len(word)
          range.append(counter)
          flag = 1
        if flag == 1 and index[1] == f:
          range.append(counter)
          break
      if range[0] != range[-1]:
        found_spaced_ill_words[word] = input[input.find(splitted_input[range[0]]): input.find(splitted_input[range[-1] + 1])]
  return found_spaced_ill_words

In [ ]:
def print_illegal_words_space(found_spaced_ill_words):
    if len(found_spaced_ill_words) > 0:
        print('\nWith white space between each word: \n')
    for w in found_spaced_ill_words:
        complete_word = found_spaced_ill_words[w]
        index = input.find(complete_word)
        print('word: \"' + w + '\"')
        print(f'span: ({index}, {index + len(complete_word)})\n')

Main function

In [ ]:
def run(input: str, illegal_words: list):
    status = 1
  
    splitted_input = input.split()

    found_ill_words = {}
    transfered_words = []
    actual_words = []
    indices = []
    counter = 0
    for word in splitted_input:
        modified_word = str(word).translate(english_trans)
        transfered_word = find_persian(modified_word)
        new_counter = counter + len(transfered_word)
        indices.append((counter, new_counter))
        transfered_words.append(transfered_word)
        counter = new_counter
        normalized_word = normalizer.normalize(transfered_word)
        lemmatized_word = lemmatizer.lemmatize(transfered_word)
        if transfered_word in illegal_words or (transfered_word in abbreviation and abbreviation[transfered_word] in illegal_words) or normalized_word in illegal_words or lemmatized_word in illegal_words:
            found_ill_words[transfered_word] = word
        founded_words = find_similar_words_in_context(transfered_word)
        for founded_word in founded_words:
            normalized_founded_word = normalizer.normalize(founded_word)
            lemmatized_founded_word = lemmatizer.lemmatize(founded_word)
            if founded_word in illegal_words or (founded_word in abbreviation and abbreviation[founded_word] in illegal_words) or normalized_founded_word in illegal_words or lemmatized_founded_word in illegal_words:
                found_ill_words[transfered_word] = word
    
    print_illegal_words(input, found_ill_words)

    if status == 1: #ignoring space
        found_spaced_ill_words = find_words_ignoring_spaces(splitted_input, transfered_words, indices)
        print_illegal_words_space(found_spaced_ill_words)

Output

In [ ]:
input = "این &تف...ن۸# گ را فروختم"
illegal_words = ['تفنگ']
run(input, illegal_words)


With white space between each word: 

word: "تفنگ"
span: (4, 16)



# TEST
-----


## Test without spaces in each word

Test 1:

In [ ]:
input = "این &تف...ن۸گ# را فروختم"
illegal_words = ['تفنگ']
run(input, illegal_words)

Without any space between each word: 

word: "تفنگ"
span: (4, 14)



Test 2:

In [ ]:
input = "با ق*aاbشق و fچ^!نگ4ال غذا خوردم."
illegal_words = ['قاشق', 'چنگال']
run(input, illegal_words)

Without any space between each word: 

word: "قاشق"
span: (3, 10)

word: "چنگال"
span: (13, 22)



## Test with space in each word

Test 1:

In [ ]:
input = "این &تف...ن۸# گ را به اح* مد تو سنگ7%#3!8..اپور   فروختم"
illegal_words = ['سنگاپور', 'تفنگ', 'احمد']
run(input, illegal_words)

Without any space between each word: 

word: "سنگاپور"
span: (32, 47)


With white space between each word: 

word: "تفنگ"
span: (4, 16)

word: "احمد"
span: (22, 29)



In [ ]:
input = "من دیروز به ا8یت۳۲ ا ل یا رفتم و با رض 49ا دیدار کردم"
illegal_words = ['ایتالیا', 'رضا']
run(input, illegal_words)


With white space between each word: 

word: "ایتالیا"
span: (12, 26)

word: "رضا"
span: (36, 43)



## Test with normalizer

In [ ]:
input = "این &دیــ...#گران را فروختم"
illegal_words = ['دیگران']
run(input, illegal_words)

Without any space between each word: 

word: "دیــگران"
span: (4, 17)



## Test with lemmatizer

In [ ]:
input = "این &گ..ل.ه#ا را فروختم"
illegal_words = ['گل']
run(input, illegal_words)

Without any space between each word: 

word: "گلها"
span: (4, 13)



## Test with abbreviation in text

In [ ]:
input = "این &آ...ج#ا آپ فروختم"
illegal_words = ['ارتش جمهوری اسلامی ایران']
run(input, illegal_words)

Without any space between each word: 

word: "آجا"
span: (4, 12)



In [ ]:
input = "این &آ...ج#ا آ..پ# فروختم"
illegal_words = ['آسان پرداخت', 'ارتش جمهوری اسلامی ایران']
run(input, illegal_words)

Without any space between each word: 

word: "آجا"
span: (4, 12)

word: "آپ"
span: (13, 18)



## Test with context

In [ ]:
input = "این &س...ل#ام را فروختم"
illegal_words = ['درود']
run(input, illegal_words)

Without any space between each word: 

word: "سلام"
span: (4, 13)

